In [37]:
import urllib.request
import urllib
from bs4 import BeautifulSoup,Comment
import requests
import pandas as pd
import time

In [60]:
def search_housing(housing="茂名坊",city='上海'):
    post_params = {
        "forward": "朝向",
        "buildareatxt":"面积",
        "floortxt": "所在楼层",
        "totalfloortxt": "总楼层",
        "category":"" ,
        "strnewcodefast": "",
        "strcityfast": city,
        "projname": housing,
        "housetypes": "",
        "ablepg":"" ,
        "boolnewsearch": True,
        "pingguprice":"" ,
        "totalprice": "",
        "totalprice1": "",
        "newcode":"" ,
        "pgid":"" ,
        "_csrf": "abQHLcTE-4E5slrAPsN4Kj-Txu-z5EBXbrCM",
                  }
    post_args = urllib.parse.urlencode(post_params).encode('UTF-8')
    url = "https://fangjia.fang.com/fangjia/suggestion/transfer/sh?"+post_args.decode('utf-8')
    res = urllib.request.urlopen(url)
    soup = BeautifulSoup(res.read().decode('utf-8',errors='ignore'))

    detail_url = "https:"+soup.find('a',attrs={'class':"assess_most"}).attrs['href'][:-1]

    req = urllib.request.Request(detail_url,headers={"User-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36",
                                                     "upgrade-insecure-requests":"1",
                                                     "accept-encoding": "gzip, deflate, br",
                                                        "accept-language": "zh-CN,zh;q=0.9"
                                                    })



    res_detail= requests.request('post',detail_url,headers={"User-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.90 Safari/537.36",
                                                     "upgrade-insecure-requests":"1",
                                                     "accept-encoding": "gzip, deflate, br",
                                                        "accept-language": "zh-CN,zh;q=0.9"
                                                    })



    soup = BeautifulSoup(res_detail.content.decode('GBK'),'html.parser')
    
#     return soup

    ## 过滤html中的comments
    comments = soup.findAll(text=lambda text: isinstance(text, Comment))
    # nit: It isn't good practice to use a list comprehension only for its
    # side-effects. (Wastes space constructing an unused list)
    for comment in comments:
        comment.extract()

    info_items = soup.find("div",attrs={'class':"Rinfolist"}).find_all('li')

    strs = {item.contents[0].get_text():item.contents[1] for item in info_items if isinstance(item.contents[1],str) and item.contents[1]!='\n'}
    tags = {item.contents[0].get_text():item.get_text().replace(item.contents[0].get_text(),"").strip("\n").replace("\n",';') for item in info_items if not isinstance(item.contents[1],str) or item.contents[1]=='\n'}
    infos = {**strs,**tags}

    infos['均价(元)'] = soup.find('span',attrs={'class':"prib"}).get_text()

    return infos,detail_url

In [ ]:
# search_housing("茂名坊")

In [9]:
# search_housing("凯德嘉博名邸")

In [1]:
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

In [26]:
driver = webdriver.Remote(
   command_executor='http://selenium:4444/wd/hub',
   desired_capabilities=DesiredCapabilities.CHROME)

In [27]:
driver.get('https://maomingfang.fang.com')

In [57]:
def get_surrounding(url):
    driver.get(url)
    iframe_tag = driver.find_element_by_id("iframe_map")
    driver.switch_to.frame(iframe_tag)
    soup_map = BeautifulSoup(driver.page_source)
    tags = soup_map.find_all('div',attrs={'class':'title1'})
    info = [{"住宅":t.find('span').get_text(),
             "距离":t.find('b').get_text()} for t in tags]
    return info

In [61]:
def query_housing(housing,city='上海'):
    subj_info,subj_url = search_housing(housing,city)
    subj_info['住宅'] = housing
    subj_info['距离'] = "0米"
    l = [subj_info]
    surroundings = get_surrounding(subj_url)
    for sur_basic in surroundings:
        print(sur_basic)
        try:
            sur_info,sur_url = search_housing(sur_basic['住宅'])
            sur_info = {**sur_info,**sur_basic}
            l.append(sur_info)
        except Exception as e:
            print(e)
        time.sleep(1)
    df = pd.DataFrame.from_dict(l)
    df = df.applymap(str)
    return df


In [62]:
df = query_housing("华府天地")

{'住宅': '翠湖天地御苑', '距离': '399米'}
{'住宅': '翠湖天地隽荟', '距离': '293米'}
HTTP Error 403: Forbidden
{'住宅': '翠湖天地嘉苑', '距离': '205米'}
{'住宅': '复兴珑御', '距离': '1055米'}
HTTP Error 403: Forbidden
{'住宅': '翠湖天地雅苑', '距离': '316米'}
{'住宅': '汇龙新城', '距离': '1191米'}
'gbk' codec can't decode byte 0xab in position 85221: illegal multibyte sequence
{'住宅': '露香园', '距离': '1252米'}
HTTP Error 403: Forbidden
{'住宅': '中凯城市之光', '距离': '1218米'}
{'住宅': '黄浦新苑', '距离': '1362米'}
{'住宅': '老西门新苑', '距离': '951米'}
{'住宅': '嘉御庭', '距离': '1867米'}
HTTP Error 403: Forbidden
{'住宅': '凯德茂名公馆', '距离': '1291米'}
{'住宅': '永业公寓', '距离': '1216米'}
{'住宅': '巴卡拉', '距离': '412米'}
{'住宅': '淡水湾花园', '距离': '960米'}
{'住宅': '思南公馆东苑', '距离': '735米'}
{'住宅': '上海天玺', '距离': '1622米'}
{'住宅': '士林华苑', '距离': '1712米'}


In [52]:
df.to_excel("华府天地及周边小区.xlsx",index=False,encoding='utf-8')